## Анализ логов продуктового магазина

Представьте, что вы работаете аналитиком онлайн-магазина и ваша цель - повысить GMV (суммарную выручку от продажи товаров). Для этого вы хотите провести ряд исследований и придумать какие-то меры для увеличения выручки.


В файле <i>orders_with_items.json</i> представлена информация о заказах в онлайн-магазине.
```
OrderID - ID Заказа,
UserID - ID клиента,
OrderTime - время заказа,
items - список элементов в заказе, каждый элемент описан словарем:
    ItemID  - ID продукта, 
    price - цена единицы товара, 
    quantity - количество единиц товара в заказе
```
В файле <i>items_with_categ.csv</i> представлена информация о категории товаров.
```
ItemID - ID продукта,
Category - категория продукта,
ParentCategory - более крупная категория продукта
```

1. Для начала поймем, какого числа было сделано больше всего заказов?
2. Давайте посмотрим на категории товаров. Найдите топ-3 ParentCategory по GMV (суммарная выручка, которую принесла продажа товаров этой категории)?
3. Представьте, что мы хотим запустить какую-то акцию для клиентов или предложить какую-то бонусную программу. Какую акцию было бы целесообразно провести и на каких клиентов её ориентировать? Составьте какой-то список клиентов, которым вы бы предложили участие в акции/программе.
4. Представьте, что для повышения GMV мы хотим внедрить на сайте систему рекомендаций и предлагать пользователю какие-то товары по мере сбора корзины. Что бы вы порекомендовали пользователю X?

Если для решения задачи вам потребуются какие-то сторонние библиотеки, вы можете их установить. __[Документация по установке пакетов](https://cloud.yandex.ru/docs/datasphere/operations/projects/install-dependencies)__

In [34]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
df = pd.read_csv("data/items_with_categ.csv")
df.head(20)

,ItemID,Category,ParentCategory
0,1000002,Сладости для диабетиков,Сладости
1,1000046,Вода,Напитки
2,1000048,Сладости для диабетиков,Сладости
3,1000049,Вода,Напитки
4,1000061,Фрукты,Овощи и фрукты
5,1000062,Фрукты,Овощи и фрукты
6,1000064,Фрукты,Овощи и фрукты
7,1000066,Фрукты,Овощи и фрукты
8,1000076,Мыло,"Красота, гигиена, здоровье"
9,1000085,Товары для праздника,Для праздника


In [48]:
df_orders = pd.read_json("data/orders_with_items.json")
df_orders

,OrderID,UserID,OrderTime,items
0,90659865,1.609460e+18,2021-01-01 03:51:42,"[{'ItemID': '413736', 'price': 36.72, 'quantit..."
1,90622054,2.499293e+18,2021-01-01 04:13:13,"[{'ItemID': '961454', 'price': 179.9, 'quantit..."
2,90659952,3.945866e+17,2021-01-01 05:48:53,"[{'ItemID': '306553', 'price': 101.9, 'quantit..."
3,89765281,5.412765e+18,2021-01-01 08:33:41,"[{'ItemID': '979703', 'price': 99.9, 'quantity..."
4,90582308,6.380061e+18,2021-01-01 09:55:35,"[{'ItemID': '303765', 'price': 44.9, 'quantity..."
...,...,...,...,...
40370,93920417,8.412061e+18,2021-06-30 23:40:23,"[{'ItemID': '310971', 'price': 77.49, 'quantit..."
40371,93924602,7.464423e+18,2021-06-30 23:41:41,"[{'ItemID': '509361', 'price': 79.9, 'quantity..."
40372,93952166,1.605989e+18,2021-06-30 23:49:12,"[{'ItemID': '444839', 'price': 129.9, 'quantit..."
40373,93955735,7.254769e+18,2021-06-30 23:56:21,"[{'ItemID': '875741', 'price': 49.0, 'quantity..."


In [53]:
type(df_orders['items'])

pandas.core.series.Series

1. Для начала поймем, какого числа было сделано больше всего заказов?

In [37]:
df_orders['date'] = pd.to_datetime(df_orders['OrderTime']).apply(lambda d: d.date())


In [38]:
df_orders['date'].value_counts().sort_values(ascending=False)

2021-02-05    342
2021-03-06    335
2021-02-12    321
2021-01-16    318
2021-02-13    313
             ... 
2021-06-06    153
2021-05-23    149
2021-06-14    148
2021-05-02    124
2021-01-01     75
Name: date, Length: 181, dtype: int64

2. Давайте посмотрим на категории товаров. Найдите топ-3 ParentCategory по GMV (суммарная выручка, которую принесла продажа товаров этой категории)?


In [ ]:

pd.DataFrame()

full_df = pd.DataFrame()

for index, order in df_orders.sample(100).iterrows():
    df_items = pd.DataFrame(order['items'])
    
    df_items['OrderID'] = order['OrderID']
    df_items['UserID'] = order['UserID']
    df_items['OrderTime'] = order['OrderTime']
    df_items['date'] = order['date']
    
    full_df = pd.concat([full_df, df_items])
    
full_df = full_df.reset_index(drop=True)
full_df

In [ ]:

pd.DataFrame()

In [39]:
full_df = pd.DataFrame()

for index, order in df_orders.sample(100).iterrows():
    df_items = pd.DataFrame(order['items'])
    
    df_items['OrderID'] = order['OrderID']
    df_items['UserID'] = order['UserID']
    df_items['OrderTime'] = order['OrderTime']
    df_items['date'] = order['date']
    
    full_df = pd.concat([full_df, df_items])
    
full_df = full_df.reset_index(drop=True)
full_df

,ItemID,price,quantity,OrderID,UserID,OrderTime,date
0,706156,245.56,2,93672987,1.002074e+18,2021-06-13 14:22:58,2021-06-13
1,312737,44.96,1,93672987,1.002074e+18,2021-06-13 14:22:58,2021-06-13
2,341219,106.16,8,93672987,1.002074e+18,2021-06-13 14:22:58,2021-06-13
3,692247,101.98,1,93672987,1.002074e+18,2021-06-13 14:22:58,2021-06-13
4,310466,96.90,4,93672987,1.002074e+18,2021-06-13 14:22:58,2021-06-13
...,...,...,...,...,...,...,...
2239,308362,289.90,2,90798873,3.222832e+18,2021-01-15 19:38:04,2021-01-15
2240,740128,176.31,1,90798873,3.222832e+18,2021-01-15 19:38:04,2021-01-15
2241,315171,126.81,1,90798873,3.222832e+18,2021-01-15 19:38:04,2021-01-15
2242,314858,179.91,1,90798873,3.222832e+18,2021-01-15 19:38:04,2021-01-15


In [40]:
full_df['ItemID'] = full_df['ItemID'].apply(int)

full_df = full_df.merge(df, left_on='ItemID', right_on='ItemID')

In [41]:
full_df['full_price'] = full_df['price'] * full_df['quantity']
full_df

,ItemID,price,quantity,OrderID,UserID,OrderTime,date,Category,ParentCategory,full_price
0,706156,245.56,2,93672987,1.002074e+18,2021-06-13 14:22:58,2021-06-13,Сладости для диабетиков,Сладости,491.12
1,312737,44.96,1,93672987,1.002074e+18,2021-06-13 14:22:58,2021-06-13,Специи и приправы,Бакалея,44.96
2,341219,106.16,8,93672987,1.002074e+18,2021-06-13 14:22:58,2021-06-13,Вода,Напитки,849.28
3,692247,101.98,1,93672987,1.002074e+18,2021-06-13 14:22:58,2021-06-13,Фрукты,Овощи и фрукты,101.98
4,692247,125.98,1,92865776,7.765680e+18,2021-04-26 21:02:37,2021-04-26,Фрукты,Овощи и фрукты,125.98
...,...,...,...,...,...,...,...,...,...,...
2239,307965,89.91,1,90798873,3.222832e+18,2021-01-15 19:38:04,2021-01-15,"Лаваши, лепешки",Мучные изделия,89.91
2240,314636,119.90,1,90798873,3.222832e+18,2021-01-15 19:38:04,2021-01-15,Фрукты,Овощи и фрукты,119.90
2241,308362,289.90,2,90798873,3.222832e+18,2021-01-15 19:38:04,2021-01-15,Сыры,Молочная продукция,579.80
2242,740128,176.31,1,90798873,3.222832e+18,2021-01-15 19:38:04,2021-01-15,"Горчица, хрен",Бакалея,176.31


In [45]:
full_df.groupby('ParentCategory').sum()
# ['full_price'].sort_values(ascending=False).head(3)

,ItemID,price,quantity,OrderID,UserID,full_price
ParentCategory,,,,,,
Бакалея,173364005,45202.79,596,35859452928,1.728228e+21,62360.05
Бытовая химия,30360294,11000.91,62,5072110844,2.405613e+20,13463.81
Для детей,6517885,3615.65,60,1096793509,7.453722e+19,6401.94
Для дома и сада,8505026,1922.00,19,1483209038,6.898604e+19,2299.80
Для праздника,5738748,2017.00,18,1110683739,5.106335e+19,2378.40
Зоотовары,17302149,9397.69,139,2404108239,1.215318e+20,14735.00
"Красота, гигиена, здоровье",46595182,12583.53,120,8658508042,3.837980e+20,14518.90
Кухонные принадлежности,10408298,8079.83,21,1755758726,1.001953e+20,8539.63
Молочная продукция,202172543,41824.97,1158,38168015368,1.566286e+21,84178.43
